In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import torch

In [ ]:
torch.cuda.is_available()

In [ ]:
# misc
import os
from tqdm import tqdm, tnrange, tqdm_notebook
from pathlib import Path
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys
import scipy.ndimage

In [ ]:
from fastai.vision import *
from fastai.metrics import *
from fastai.callbacks import *

## Define paths

In [ ]:
path_str = '/home/ubuntu/sfr-challenge/lungs/dataset/Inference'

In [ ]:
path_model_str = '/home/ubuntu/sfr-challenge/lungs/dataset/train'

In [ ]:
path = Path(path_str)
os.path.exists(path)

In [ ]:
path_model = Path(path_model_str)
os.path.exists(path_model)

## Define fixed random seed

In [ ]:
np.random.seed(42)

## Inference loop

In [ ]:
def get_prediction_glob(preds):
    max_pred = 0
    for n in preds[:, 1]:
        if max_pred < n:
            max_pred = n
    return float(max_pred)

In [ ]:
def get_prediction_glob_av(preds):
    max_1 = 0.0
    max_2 = 0.0
    for n in preds[:, 1]:
        if max_1 < n:
            max_1 = float(n)
        elif max_1 >= n and max_2 < n:
            max_2 = float(n)
    return (max_1 + max_2)/2

In [ ]:
dirs = os.listdir(path/'Patients')
len(dirs)

In [ ]:
preds = []
patients = []
preds_glob = []
all_preds = []
detailed_preds = {}
error = []
for d in tqdm(dirs):
    learner = load_learner(path_model, 'final-vgg16-12epc-1013-1130', test=ImageList.from_folder(path/'Patients'/d)).to_fp16()
    try:
        preds, y = learner.get_preds(ds_type=DatasetType.Test)
        preds_glob.append(get_prediction_glob(preds))
        patients.append(d)
        detailed_preds[d] = preds
    except:
        error.append(d)
        print('patient is invalid: ', d)

In [ ]:
# must 344 patients
print("length patients: ", len(patients))
print("length preds: ", len(preds_glob))
print("samples patients: ", patients[:3])
print("samples predicts: ", preds_glob[:3])

### Re-generate file with predictions base on modified predictions function

In [ ]:
patients = []
preds_glob = []
for key in detailed_preds:
    patients.append(key)
    preds_glob.append(get_prediction_glob(detailed_preds[key]))

## Create output .csv file

In [ ]:
inference_output = pd.DataFrame({'examen' : patients, 'probabilité' : preds_glob})
inference_output.set_index('examen', inplace=True)
inference_output.to_csv(path/'NAIS.csv',sep=';',decimal=',')